# Lab 1: Introduction to Machine Learning with scikit-learn

**Principles of AI** · IIT Gandhinagar

---

## Lab Overview

In this lab, you will learn the fundamentals of machine learning using scikit-learn (sklearn), Python's most popular ML library.

**By the end of this lab, you will be able to:**
1. Load and explore datasets
2. Understand the difference between features and labels
3. Split data into training and testing sets
4. Train your first classifier (Logistic Regression)
5. Evaluate model performance using accuracy and confusion matrices

**Duration**: 2-3 hours

---

## Setup

First, let's install and import the required libraries.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install scikit-learn pandas matplotlib seaborn

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn imports
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("All imports successful!")

---

# Part 1: Understanding Data

## 1.1 What is a Dataset?

In machine learning, a **dataset** is a collection of examples that we use to train our models.

```
┌─────────────────────────────────────────────────────────────┐
│                        DATASET                              │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   Each ROW = One example (sample)                           │
│   Each COLUMN = One feature (attribute)                     │
│   One special column = The LABEL (what we predict)          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Example**: If we're classifying flowers:
- **Features**: petal length, petal width, sepal length, sepal width
- **Label**: flower species (setosa, versicolor, virginica)

## 1.2 Loading the Iris Dataset (Solved Example)

The **Iris dataset** is one of the most famous datasets in ML. It contains measurements of 150 iris flowers from 3 different species.

In [ ]:
# SOLVED EXAMPLE: Loading and exploring the Iris dataset

# Load the dataset
iris = load_iris()

# What's inside?
print("Keys in the dataset:", iris.keys())
print("\n" + "="*50)
print("Feature names:", iris.feature_names)
print("Target names (classes):", iris.target_names)
print("\n" + "="*50)
print("Shape of features (X):", iris.data.shape)  # 150 samples, 4 features
print("Shape of labels (y):", iris.target.shape)   # 150 labels

In [ ]:
# Convert to pandas DataFrame for easier viewing
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = [iris.target_names[i] for i in iris.target]

print("First 10 rows of the dataset:")
df.head(10)

In [ ]:
# Basic statistics
print("Dataset statistics:")
df.describe()

In [ ]:
# How many of each species?
print("Class distribution:")
df['species'].value_counts()

## 1.3 Visualizing the Data

Visualization helps us understand patterns in the data before training.

In [ ]:
# SOLVED EXAMPLE: Scatter plot of two features

plt.figure(figsize=(10, 6))

# Plot each species with a different color
colors = {'setosa': 'red', 'versicolor': 'blue', 'virginica': 'green'}
for species in iris.target_names:
    subset = df[df['species'] == species]
    plt.scatter(subset['sepal length (cm)'], 
                subset['petal length (cm)'],
                c=colors[species], 
                label=species,
                alpha=0.7,
                s=50)

plt.xlabel('Sepal Length (cm)')
plt.ylabel('Petal Length (cm)')
plt.title('Iris Dataset: Sepal Length vs Petal Length')
plt.legend()
plt.show()

print("\n💡 Notice how the species form distinct clusters!")
print("   This suggests a classifier should work well on this data.")

### Question 1.1: Explore Feature Relationships

Create a scatter plot of **petal length** vs **petal width**. Do the species still form distinct clusters?

In [ ]:
# YOUR CODE HERE
# Create a scatter plot similar to the solved example
# but use 'petal length (cm)' and 'petal width (cm)'



### Question 1.2: Pair Plot

Use seaborn's `pairplot` to visualize ALL pairwise relationships at once.

**Hint**: `sns.pairplot(df, hue='species')`

In [ ]:
# YOUR CODE HERE



---

# Part 2: Preparing Data for Machine Learning

## 2.1 Why Split Data?

```
┌─────────────────────────────────────────────────────────────┐
│                     THE EXAM ANALOGY                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   Training Data = Practice problems you study from          │
│   Test Data = The actual exam (unseen problems)             │
│                                                             │
│   If you memorize practice problems word-for-word,          │
│   you might fail on new problems in the exam!               │
│                                                             │
│   Same with ML: We need UNSEEN data to test true learning.  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Key Insight**: We NEVER train on test data. The test set simulates "real world" data the model hasn't seen.

## 2.2 Train-Test Split (Solved Example)

In [ ]:
# SOLVED EXAMPLE: Splitting data into train and test sets

# Get features (X) and labels (y)
X = iris.data  # All 4 features
y = iris.target  # The species labels (0, 1, or 2)

print("Before split:")
print(f"  Total samples: {len(X)}")

# Split: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% for testing
    random_state=42,    # For reproducibility
    stratify=y          # Keep class proportions same in both sets
)

print("\nAfter split:")
print(f"  Training samples: {len(X_train)} ({len(X_train)/len(X)*100:.0f}%)")
print(f"  Testing samples: {len(X_test)} ({len(X_test)/len(X)*100:.0f}%)")

In [ ]:
# Verify stratification worked (same class proportions)
print("Class distribution in training set:")
unique, counts = np.unique(y_train, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  Class {u} ({iris.target_names[u]}): {c} samples")

print("\nClass distribution in test set:")
unique, counts = np.unique(y_test, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  Class {u} ({iris.target_names[u]}): {c} samples")

### Question 2.1: Different Split Ratios

Try splitting with 70-30 ratio instead of 80-20. How many samples are in each set now?

In [ ]:
# YOUR CODE HERE
# Use test_size=0.3 for 70-30 split



## 2.3 Feature Scaling

**Why scale features?**

Some algorithms (like Logistic Regression, KNN) are sensitive to the scale of features.

```
Example without scaling:
  - Feature 1: House size in sq ft (range: 500 - 5000)
  - Feature 2: Number of bedrooms (range: 1 - 6)
  
  The algorithm might think Feature 1 is more important 
  just because it has bigger numbers!
```

**StandardScaler** transforms features to have:
- Mean = 0
- Standard deviation = 1

In [ ]:
# SOLVED EXAMPLE: Feature scaling

# Before scaling
print("Before scaling:")
print(f"  Feature means: {X_train.mean(axis=0).round(2)}")
print(f"  Feature stds:  {X_train.std(axis=0).round(2)}")

# Create and fit the scaler on TRAINING data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # fit + transform
X_test_scaled = scaler.transform(X_test)        # only transform (use training stats)

print("\nAfter scaling:")
print(f"  Feature means: {X_train_scaled.mean(axis=0).round(2)}")
print(f"  Feature stds:  {X_train_scaled.std(axis=0).round(2)}")

**Important**: We fit the scaler on training data only! We use the SAME transformation parameters for test data. This prevents "data leakage" - using information from the test set during training.

---

# Part 3: Training Your First Classifier

## 3.1 What is Classification?

```
                    ┌─────────────────┐
   Input            │   Classifier    │         Output
   Features    ───► │   (Model)       │  ───►   Class Label
   [5.1, 3.5,       │                 │         "setosa"
    1.4, 0.2]       └─────────────────┘
```

The classifier learns patterns in training data, then predicts classes for new data.

## 3.2 Logistic Regression (Solved Example)

Despite its name, **Logistic Regression** is used for **classification**, not regression.

It's one of the simplest and most interpretable classifiers.

In [ ]:
# SOLVED EXAMPLE: Training a Logistic Regression classifier

# Step 1: Create the model
model = LogisticRegression(max_iter=200)  # max_iter ensures convergence

# Step 2: Train the model on training data
model.fit(X_train_scaled, y_train)

print("Model trained successfully!")
print(f"\nModel type: {type(model).__name__}")
print(f"Number of classes: {len(model.classes_)}")
print(f"Classes: {model.classes_}")

In [ ]:
# Step 3: Make predictions on test data
y_pred = model.predict(X_test_scaled)

# Compare predictions with actual labels
print("First 10 predictions vs actual:")
print("-" * 40)
for i in range(10):
    pred_name = iris.target_names[y_pred[i]]
    actual_name = iris.target_names[y_test[i]]
    match = "✓" if y_pred[i] == y_test[i] else "✗"
    print(f"  Predicted: {pred_name:12} | Actual: {actual_name:12} {match}")

In [ ]:
# Logistic Regression also gives probabilities!
y_proba = model.predict_proba(X_test_scaled)

print("Probabilities for first 5 samples:")
print("-" * 60)
print(f"{'Sample':<8} {'P(setosa)':<12} {'P(versicolor)':<14} {'P(virginica)':<12}")
print("-" * 60)
for i in range(5):
    print(f"{i:<8} {y_proba[i][0]:<12.3f} {y_proba[i][1]:<14.3f} {y_proba[i][2]:<12.3f}")

### Question 3.1: Train a K-Nearest Neighbors Classifier

KNN is another simple classifier. It predicts based on the majority class of the K nearest training examples.

Train a KNN classifier with `n_neighbors=5` and make predictions on the test set.

In [ ]:
# YOUR CODE HERE
# 1. Create KNeighborsClassifier with n_neighbors=5
# 2. Fit on X_train_scaled and y_train
# 3. Predict on X_test_scaled
# 4. Print first 10 predictions



---

# Part 4: Evaluating Your Model

## 4.1 Accuracy

**Accuracy** = (Correct predictions) / (Total predictions)

It's the simplest metric, but not always the best!

In [ ]:
# SOLVED EXAMPLE: Calculating accuracy

# Using sklearn's accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")

# Or calculate manually
correct = (y_test == y_pred).sum()
total = len(y_test)
print(f"\nManual calculation: {correct}/{total} = {correct/total:.2%}")

## 4.2 Confusion Matrix

A confusion matrix shows WHERE the model makes mistakes.

```
                    Predicted
               Setosa  Versi.  Virgin.
Actual Setosa    10      0       0
       Versi.     0      9       1
       Virgin.    0      0      10
       
Diagonal = Correct predictions
Off-diagonal = Mistakes
```

In [ ]:
# SOLVED EXAMPLE: Confusion matrix visualization

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot it
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Logistic Regression')
plt.show()

# Interpret the results
print("\n📊 Interpretation:")
print("   Diagonal values = Correct predictions")
print("   Off-diagonal = Errors (confusion between classes)")

In [ ]:
# Detailed classification report
print("Classification Report:")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=iris.target_names))

### Question 4.1: Compare Models

Calculate the accuracy and create a confusion matrix for your KNN classifier from Question 3.1.

Which model performs better: Logistic Regression or KNN?

In [ ]:
# YOUR CODE HERE
# 1. Calculate accuracy for KNN predictions
# 2. Create and plot confusion matrix
# 3. Compare with Logistic Regression



## 4.3 When Accuracy Lies

**Warning**: Accuracy can be misleading for imbalanced datasets!

```
Example: Disease detection with 99% healthy, 1% sick

"Dumb" model: Always predict "healthy"
Accuracy = 99% (Looks great!)

But it NEVER detects sick patients! That's dangerous.
```

In [ ]:
# Demonstration: Accuracy on imbalanced data

# Create fake imbalanced data
y_imbalanced = np.array([0]*99 + [1]*1)  # 99 class-0, 1 class-1
y_dumb_pred = np.zeros(100)  # Always predict class 0

print("Imbalanced dataset example:")
print(f"  Class 0: 99 samples")
print(f"  Class 1: 1 sample")
print(f"\n'Always predict 0' accuracy: {accuracy_score(y_imbalanced, y_dumb_pred):.0%}")
print("\n⚠️ The model is useless but has 99% accuracy!")

---

# Part 5: Working with the Digits Dataset

Now let's try a more challenging dataset: handwritten digits (similar to MNIST).

In [ ]:
# Load the digits dataset
digits = load_digits()

print("Digits dataset info:")
print(f"  Number of samples: {len(digits.data)}")
print(f"  Number of features: {digits.data.shape[1]}")
print(f"  Number of classes: {len(digits.target_names)}")
print(f"  Classes: {digits.target_names}")
print(f"\nEach digit is an 8x8 pixel image (64 features)")

In [ ]:
# Visualize some digits
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='gray')
    ax.set_title(f"Label: {digits.target[i]}")
    ax.axis('off')

plt.suptitle('Sample Handwritten Digits', fontsize=14)
plt.tight_layout()
plt.show()

### Question 5.1: Complete Pipeline for Digits

Build a complete ML pipeline for the digits dataset:
1. Split into train/test (80/20)
2. Scale the features
3. Train a Logistic Regression classifier
4. Calculate accuracy
5. Create confusion matrix

What accuracy do you achieve? Which digits are most often confused?

In [ ]:
# YOUR CODE HERE
# Complete the pipeline step by step

# Step 1: Get X and y
X_digits = digits.data
y_digits = digits.target

# Step 2: Train-test split


# Step 3: Scale features


# Step 4: Train model


# Step 5: Predict and evaluate


# Step 6: Confusion matrix



### Question 5.2: Visualize Mistakes

Find samples where the model made mistakes and visualize them. Can you understand why the model was confused?

In [ ]:
# YOUR CODE HERE
# Hint: Find indices where y_pred != y_test
# Then visualize those digits



---

# Challenge Problems

These are optional exercises for those who finish early or want extra practice.

### Challenge 1: Hyperparameter Tuning

For KNN, try different values of `n_neighbors` (1, 3, 5, 7, 9, 11) on the Iris dataset.
Which value gives the best accuracy? Plot accuracy vs n_neighbors.

In [ ]:
# YOUR CODE HERE



### Challenge 2: Cross-Validation

Instead of a single train-test split, use 5-fold cross-validation.

**Hint**: Use `from sklearn.model_selection import cross_val_score`

In [ ]:
# YOUR CODE HERE



### Challenge 3: Load Your Own Dataset

Download a CSV dataset from Kaggle or elsewhere and build a classifier for it.

Suggestions:
- Titanic survival prediction
- Heart disease prediction
- Mushroom classification

In [ ]:
# YOUR CODE HERE



---

# Summary

In this lab, you learned:

1. **Datasets** have features (X) and labels (y)
2. **Train-test split** prevents overfitting and tests generalization
3. **Feature scaling** helps many algorithms perform better
4. **Logistic Regression** and **KNN** are simple but effective classifiers
5. **Accuracy** measures overall correctness but can be misleading
6. **Confusion matrices** show which classes are confused

## Key Code Summary

```python
# Load data
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Predict & Evaluate
y_pred = model.predict(X_test_scaled)
from sklearn.metrics import accuracy_score
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
```

## Next Lab

**Lab 2: Decision Trees, Random Forests, Regression & Clustering**

We'll explore more algorithms that handle different types of problems!